In [1]:
import sys
sys.path.append("..")

In [2]:
from db.duckdb.duckdbhelper import DuckDBDatabaseHelper
from pprint import pprint
import pandas as pd

In [3]:
NUM_ROWS = 1500000

In [4]:
db = DuckDBDatabaseHelper("../meters.db")

In [5]:
sql_query ="""

SELECT day,energy_sum, 
temperatureMin, 
temperatureMax,
uvIndex,
dewPoint,                  
Type,
date_part('month', day) as month,
m.LCLid
FROM  meters m LEFT JOIN info_household ih
ON m.LCLid = ih.LCLid
LEFT JOIN weather_daily_darksky_modified wd
ON m.day = wd.formatted_temperatureMaxTime_weather_daily_darksky
LEFT JOIN uk_bank_holidays uk_bh
ON m.day = uk_bh."Bank holidays"

WHERE date_part('year', day) = 2014
AND date_part('month', day) = 1

"""

db.connect()
records = db.fetch_all(sql_query)
db.close_connection()

Connected to DuckDB database: ../meters.db
Fetched 157488 rows.
Connection closed.


In [6]:
records[:1]

[(datetime.date(2014, 1, 2),
  22.529999899999996,
  6.54,
  10.75,
  1.0,
  6.02,
  None,
  1,
  'MAC003223')]

In [7]:
COLS = [
"day","energy_sum", 
"temperatureMin", 
"temperatureMax",
"uvIndex",
"dewPoint",                  
"Type","month","LCLid" ]

In [8]:
df_model = pd.DataFrame(records,columns = COLS)

In [9]:
df_model.head()

,day,energy_sum,temperatureMin,temperatureMax,uvIndex,dewPoint,Type,month,LCLid
0,2014-01-02,22.530,6.54,10.75,1.0,6.02,None,1,MAC003223
1,2014-01-03,19.105,7.01,10.49,1.0,5.08,None,1,MAC003223
2,2014-01-06,24.482,9.34,12.57,1.0,7.64,None,1,MAC003223
3,2014-01-07,23.529,8.98,11.38,1.0,6.63,None,1,MAC003223
4,2014-01-08,18.502,7.37,11.62,1.0,7.02,None,1,MAC003223


In [10]:
df_model2 = df_model[df_model["LCLid"] == 'MAC000002']

In [11]:
df_model2['Type_Binary'] = df_model2['Type'].apply(lambda x: 1 if x is not None else 0)

C:\Users\Ambarish Ganguly\AppData\Local\Temp\ipykernel_17688\3940123378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model2['Type_Binary'] = df_model2['Type'].apply(lambda x: 1 if x is not None else 0)


In [12]:
len(df_model2)

31

In [13]:
df_model2.columns

Index(['day', 'energy_sum', 'temperatureMin', 'temperatureMax', 'uvIndex',
       'dewPoint', 'Type', 'month', 'LCLid', 'Type_Binary'],
      dtype='object')

In [14]:
df_model2.to_csv("../model_data/test_day.csv",index = False)